# Visualisation of LFP & Sleep scoring

### Load LFP and packages

In [1]:
cd "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/"

\\10.69.168.1\crnldata\waking\audrey_hay\L1imaging\AnalysedMarch2023


In [2]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer
from scipy.stats import zscore
from scipy.interpolate import interp1d
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
import ephyviewer

from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

%matplotlib widget

In [44]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = False, title = "<b>Open Ephys folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='\\10.69.168.1\crnldata\waking\audrey_hay', filename='', title='<b>Open Ephys folder</b>', sh…

Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)


In [64]:
folder_base = Path(dpath) 
DataRec = np.fromfile(folder_base, dtype="int16")
All = DataRec.reshape(-1,32)
All.shape

DataRec2 = np.fromfile(folder_base, dtype="uint16")
All2 = DataRec2.reshape(-1,32)
All2.shape

Channels = '//10.69.168.1/crnldata/waking/audrey_hay/AurelieB/LFPChannels_perMice.xlsx' 
mice=folder_base.parts[-1].split('_')[0]
print(mice)

allchannels = pd.read_excel(Channels)

PFCch1=int(allchannels[mice][0].split(',')[0])
PFCch2=int(allchannels[mice][0].split(',')[1])
CA1ch1=int(allchannels[mice][2].split(',')[0])
CA1ch2=int(allchannels[mice][2].split(',')[1])
S1ch1=int(allchannels[mice][1].split(',')[0])
S1ch2=int(allchannels[mice][1].split(',')[1])
EMGch1=int(allchannels[mice][3])

PFC  =  All[:, PFCch1]-All[:, PFCch2] 
CA1  =  All[:, CA1ch1]-All[:, CA1ch2] 
S1  =  All[:, S1ch1]-All[:, S1ch2] 
EMG  =  All2[:, EMGch1]


"""
PFC  =  All2[:, PFCch1]
CA1  =  All2[:, CA1ch1]
S1  =  All2[:, S1ch1]
PFC2  =  All2[:, PFCch2] 
CA12  =  All2[:, CA1ch2] 
S12  =  All2[:, S1ch2] 
EMG  =  All2[:, EMGch1]
"""
combined = np.stack([zscore(S1),zscore(PFC), zscore(CA1), zscore(EMG)], axis = 1)
#combined = np.stack([zscore(S1),zscore(S12),zscore(PFC),zscore(PFC2), zscore(CA1),zscore(CA12), zscore(EMG)], axis = 1)

Lou


# EphyViewer


In [65]:
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer, CsvEpochSource, EpochEncoder,EpochViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import InMemorySpikeSource

app = mkQApp()
win = MainViewer(debug=True, show_auto_scale=True)

# LFPs

sample_rate = 1000
t_start = 0.

#source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222', 2: '#FFFFFF', 3: '#222222', 4: '#FFFFFF', 5: '#222222'}, channel_names=['S1','PFC', 'CA1', 'EMG', 'Scoring'])
source =InMemoryAnalogSignalSource(combined, sample_rate, t_start, channel_names=['S1','PFC', 'CA1', 'EMG', 'Scoring'])
view1 = TraceViewer(source=source)

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.by_channel_params['ch0', 'color'] = '#88FF88' #FF0000 red, #00FF00 green, and #0000FF blue
view1.by_channel_params['ch1', 'color'] = '#8888FF'
view1.by_channel_params['ch2', 'color'] = '#FF8888'
view1.by_channel_params['ch3', 'color'] = '#FFFFFF'


# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 250
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz

view3.by_channel_params['ch0', 'clim'] = 1
view3.by_channel_params['ch1', 'clim'] = 1
view3.by_channel_params['ch2', 'clim'] = 1
view3.by_channel_params['ch0', 'visible'] = True
view3.by_channel_params['ch1', 'visible'] = True
view3.by_channel_params['ch2', 'visible'] = True
view3.by_channel_params['ch3', 'visible'] = False


win.add_view(view1)
win.add_view(view3)

#Run

win.show()
app.exec_()

debug True
QT_MODE PyQt5
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for FFT 0.0 s
refresh duration for  0.0 s
refresh duration for FFT 0.0009975433349609375 s
refresh duration for  0.0 s
refresh duration for FFT 0.0 s
refresh duration for  0.0 s
refresh duration for FFT 0.0 s
refresh duration for  0.0 s
refresh duration for FFT 0.0 s
refresh duration for  0.0 s
refresh duration for FFT 0.0 s
refresh duration for  0.0 s
refresh duration for FFT 0.0 s
refresh duration for  0.0 s
refresh duration for FFT 0.0009996891021728516 s
refresh duration for  0.0 s
refresh duration for FFT 0.0009975433349609375 s
viewer has moved already 1 27.250431999999996 4.954624
viewer has moved already 1 27.250431999999996 9.909248
viewer has moved already 1 27.250431999999996 12.38656
viewer has moved already 2 27.250431999999996 4.954624
viewer has moved already 2 27.250431999999996 9.909248
viewer has moved already 2 27.250431999999996 12.38656
viewer has moved already 2 27.25

0